# Effect of UBI on labor supply in OG-USA small open economy

Testing the income effect.

Goals:

* What's the implied income effect when people get a \$1,000-per-person-per-year UBI, in a small open economy?
* How does this vary with frisch and epsilon?
* How does this compare to microeconometric studies on the income effect?
* How does it compare to other models like CBO and PWBM?
* How does it vary with lifetime income level and age?

Intermediate questions:
* What is the range of the implied UBI based on its ETR effect? Is it roughly the `taxcalc`-supplied UBI value multiplied by the number of people per tax unit?
* What's the overall labor response to the \$1,000 UBI across lifetime income groups and age?

## Setup

In [1]:
import pandas as pd
import numpy as np
import ogusa
import os
import microdf as mdf

In [2]:
PATH = 'default'
SS_VARS_FILE = 'SS/SS_vars.pkl'
base = pd.read_pickle(os.path.join(PATH, 'OUTPUT_BASELINE', SS_VARS_FILE))
base_params = pd.read_pickle(os.path.join(PATH, 'OUTPUT_BASELINE/model_params.pkl'))
reform = pd.read_pickle(os.path.join(PATH, 'OUTPUT_REFORM', SS_VARS_FILE))
reform_params = pd.read_pickle(os.path.join(PATH, 'OUTPUT_REFORM/model_params.pkl'))

## Utilities

In [3]:
# Map each skill level to a printed value and its total population weight.
# Un-nest lambdas.
lambdas = [i[0] for i in base_params.lambdas.tolist()]
# Create DataFrame.
LIFETIME_INCOME_BUCKETS = ['0-25%', '25-50%', '50-70%', '70-80%', '80-90%',
                           '90-99%', 'Top 1%']
lifetime_income_map = pd.DataFrame({'skill': np.arange(7),
                                    'lifetime_income_group': LIFETIME_INCOME_BUCKETS,
                                    'lifetime_income_pop_share': lambdas})

In [4]:
# Map each age to its population weight.
omegas_ss = reform_params.omega[-1, :]  # Final time period.
age_pop_map = pd.DataFrame({'age': np.arange(21, 101),
                            'age_pop_share': omegas_ss})

In [5]:
def age_skill_df(mats, colnames):
    '''
    Produces DataFrame by age and lifetime income group
    based on 2d numpy arrays for a given value.
    
    Args:
        mats: List of 2d JxS numpy array. Can also be a singleton.
        colnames: List of column names the value should take.
            Can also be a singleton.
        
    Returns:
        DataFrame with columns for age, lifetime_income_group, and
        <colnames>.
    '''
    # Make args lists if they're not already.
    if type(mats) != list:
        mats = [mats]
        colnames = [colnames]
    df_l = []
    num = len(mats)
    assert len(colnames) == num
    for i in range(num):
        # Melt and rename.
        tmp = pd.DataFrame(mats[i]).reset_index().melt('index')
        tmp.columns = ['age', 'skill', colnames[i]]
        tmp.set_index(['age', 'skill'], inplace=True)
        df_l.append(tmp)
    # Merge all DataFrames.
    df = pd.concat(df_l, axis=1).reset_index()
    # Age starts at 21.
    df.age += 21
    # Map lifetime_income_group by skill index.
    df = df.merge(lifetime_income_map, on='skill')
    df = df.merge(age_pop_map, on='age')
    # Population share for SxJ is the product.
    df['pop_share'] = df.age_pop_share * df.lifetime_income_pop_share
    return df.drop('skill', axis=1)

## Create data

### UBI amounts

As implied through the ETR.

In [6]:
assert np.allclose(base['rss'], reform['rss'])
assert np.allclose(base['wss'], reform['wss'])
assert np.allclose(base['factor_ss'], reform['factor_ss'])

Specifications can come from baseline.

In [7]:
p = ogusa.Specifications(baseline=True, time_path=False)

In [8]:
etr_params_3D = np.tile(
    np.reshape(reform_params.etr_params[-1, :, :],
               (reform_params.S, 1, reform_params.etr_params.shape[2])),
    (1, reform_params.J, 1))

In [9]:
def net_tax(ss_vars, params, p):
    '''
    Calculates net tax amount for each SxJ, given steady-state variables,
    except for ETR parameters which can diverge.
    
    Args:
        ss_vars: Dict from a ss_vars.pkl.
        params: params object containing etr_params.
        p: ogusa.Specifications object.
        
    Returns:
        SxJ numpy array representing the net tax.
    '''
    etr_params_3D = np.tile(
        np.reshape(params.etr_params[-1, :, :],
                   (params.S, 1, params.etr_params.shape[2])),
        (1, params.J, 1))
    return ogusa.tax.net_taxes(
        r=ss_vars['rss'],
        w=ss_vars['wss'], 
        b=ss_vars['bssmat_s'],
        n=ss_vars['nssmat'],
        bq=ss_vars['bqssmat'],  # Bequests received.
        factor=ss_vars['factor_ss'],
        tr=ss_vars['TR_ss'],
        theta=ss_vars['theta'],
        t=None,
        j=None,
        shift=False,
        method='SS',
        e=p.e,
        etr_params=etr_params_3D,
        p=p)

In [10]:
# Calculate net tax for the baseline conditions,
# varying only the ETR, which governs the UBI.
net_tax_ubi = net_tax(base, reform_params, p)
net_tax_base = net_tax(base, base_params, p)

In [11]:
ubi = base['factor_ss'] * (net_tax_base - net_tax_ubi)

### Percent change in income

In [12]:
def y_aftertax(ss_vars, params, p):
    # Should this use yss_before_tax_mat instead?
    # What about bequests and government transfers received?
    return (ss_vars['rss'] * ss_vars['bssmat_s'] + 
            ss_vars['wss'] * p.e * ss_vars['nssmat'] -
            net_tax(ss_vars, params, p))

In [13]:
y_aftertax_base = base['factor_ss'] * y_aftertax(base, base_params, p)

### Combine into a `DataFrame`

With one row per SxJ.

In [14]:
def calc_ratios(df):
    # Calculate percentage change in after-tax income from UBI.
    df['y_diff'] = df.ubi / df.y_aftertax_base
    # Calculate percentage change in labor response.
    df['n_diff'] = df.n_reform / df.n_base - 1
    # Calculate elasticity.
    df['elasticity'] = df.n_diff / df.y_diff
    df['elasticity_display'] = df.elasticity.round(2)
    # Create variables for plotting.
    df['y_diff_pct'] = (df.y_diff * 100).round(2)
    df['n_diff_pct'] = (df.n_diff * 100).round(2)
    df['ubi_print'] = df.ubi.round(0)

In [15]:
sxj = age_skill_df([y_aftertax_base, ubi, base['nssmat'], reform['nssmat']],
                   ['y_aftertax_base', 'ubi', 'n_base', 'n_reform'])
calc_ratios(sxj)

### Aggregate by age and lifetime income

Using `pop_share`.

In [16]:
sxj

,age,y_aftertax_base,ubi,n_base,n_reform,lifetime_income_group,lifetime_income_pop_share,age_pop_share,pop_share,y_diff,n_diff,elasticity,elasticity_display,y_diff_pct,n_diff_pct,ubi_print
0,21,32754.317672,2061.039832,0.342540,0.332906,0-25%,0.25,0.015799,0.003950,0.062924,-0.028127,-0.447001,-0.45,6.29,-2.81,2061.0
1,21,23951.994641,2129.326383,0.309835,0.298567,25-50%,0.25,0.015799,0.003950,0.088900,-0.036368,-0.409094,-0.41,8.89,-3.64,2129.0
2,21,24084.292988,2131.707476,0.293090,0.282068,50-70%,0.20,0.015799,0.003160,0.088510,-0.037605,-0.424862,-0.42,8.85,-3.76,2132.0
3,21,26192.225119,2152.146564,0.287342,0.276981,70-80%,0.10,0.015799,0.001580,0.082167,-0.036059,-0.438847,-0.44,8.22,-3.61,2152.0
4,21,29949.487739,2123.014163,0.285998,0.276313,80-90%,0.10,0.015799,0.001580,0.070886,-0.033861,-0.477686,-0.48,7.09,-3.39,2123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,100,82646.160580,2596.039299,0.157197,0.153533,50-70%,0.20,0.000475,0.000095,0.031411,-0.023309,-0.742062,-0.74,3.14,-2.33,2596.0
556,100,98439.239153,3119.523432,0.154945,0.151328,70-80%,0.10,0.000475,0.000047,0.031690,-0.023342,-0.736562,-0.74,3.17,-2.33,3120.0
557,100,115007.769679,3304.341787,0.167680,0.162993,80-90%,0.10,0.000475,0.000047,0.028731,-0.027956,-0.973024,-0.97,2.87,-2.80,3304.0
558,100,132127.700514,3586.642908,0.161776,0.156586,90-99%,0.09,0.000475,0.000043,0.027145,-0.032087,-1.182049,-1.18,2.71,-3.21,3587.0


**NOT CORRECT** Need to weight it.

In [17]:
SUM_VARS = ['ubi', 'y_aftertax_base', 'n_base', 'n_reform', 'pop_share']
age = sxj.groupby('age')[SUM_VARS].sum()
lifetime_income = sxj.groupby('lifetime_income_group')[SUM_VARS].sum()
calc_ratios(age)
calc_ratios(lifetime_income)

In [18]:
lifetime_income

,ubi,y_aftertax_base,n_base,n_reform,pop_share,y_diff,n_diff,elasticity,elasticity_display,y_diff_pct,n_diff_pct,ubi_print
lifetime_income_group,,,,,,,,,,,,
0-25%,179472.782788,3.639739e+06,24.417514,23.760892,0.25,0.049309,-0.026891,-0.545363,-0.55,4.93,-2.69,179473.0
25-50%,227517.886036,4.802960e+06,24.295549,23.656934,0.25,0.047370,-0.026285,-0.554888,-0.55,4.74,-2.63,227518.0
50-70%,326796.312895,6.444962e+06,24.756081,24.060046,0.20,0.050706,-0.028116,-0.554488,-0.55,5.07,-2.81,326796.0
70-80%,403087.571345,7.946444e+06,24.650896,23.968459,0.10,0.050726,-0.027684,-0.545762,-0.55,5.07,-2.77,403088.0
80-90%,507566.191776,9.359877e+06,24.251882,23.555253,0.10,0.054228,-0.028725,-0.529704,-0.53,5.42,-2.87,507566.0
90-99%,628740.150248,1.233039e+07,22.865722,22.242030,0.09,0.050991,-0.027276,-0.534922,-0.53,5.10,-2.73,628740.0
Top 1%,971350.269340,2.271428e+07,20.760306,20.285535,0.01,0.042764,-0.022869,-0.534778,-0.53,4.28,-2.29,971350.0


## Export

In [19]:
sxj.to_csv('sxj.csv', index=False)

## Other exploratory

Change between base and reform to `yss_before_tax` matches change to `nss` in the first period,
diverges after that.

In [20]:
assert np.allclose(reform['yss_before_tax_mat'][0, :] /
                   base['yss_before_tax_mat'][0, :],
                   reform['nssmat'][0, :] /
                   base['nssmat'][0, :])

In [21]:
assert not np.allclose(reform['yss_before_tax_mat'][1:, :] /
                       base['yss_before_tax_mat'][1:, :],
                       reform['nssmat'][1:, :] /
                       base['nssmat'][1:, :])